In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import json

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.models import AE, MLP_Decoder, MLP_Encoder
from src.utils import dotdict

In [3]:
# Load json from defined directory
path = "../outputs/AE_2023-08-15_18-18-47/"

config = dotdict(json.load(open(path + "config.json")))
trained_model = AE(encoder=MLP_Encoder(config.encoder_layers), decoder=MLP_Decoder(config.decoder_layers))
trained_model.load_state_dict(torch.load(path + "model.pt"))
trained_model.eval()

RuntimeError: Error(s) in loading state_dict for AE:
	Missing key(s) in state_dict: "encoder.layers.0.0.weight", "encoder.layers.0.0.bias", "encoder.layers.1.0.weight", "encoder.layers.1.0.bias", "decoder.layers.0.0.weight", "decoder.layers.0.0.bias", "decoder.layers.1.0.weight", "decoder.layers.1.0.bias". 
	Unexpected key(s) in state_dict: "module.encoder.layers.0.0.weight", "module.encoder.layers.0.0.bias", "module.encoder.layers.1.0.weight", "module.encoder.layers.1.0.bias", "module.decoder.layers.0.0.weight", "module.decoder.layers.0.0.bias", "module.decoder.layers.1.0.weight", "module.decoder.layers.1.0.bias". 

In [ ]:
config

{'lr': 0.0001,
 'batch_size': 128,
 'epochs': 200,
 'num_workers': 4,
 'optimizer': 'Adam',
 'scheduler': 'CosineAnnealingLR',
 'device': 'cuda',
 'save_path': './checkpoints',
 'output_dir': './outputs',
 'seed': 1337}

In [ ]:
reconstructions = trained_model.reconstruct(eval_dataset[:25].to(device)).detach().cpu()

In [ ]:
# show reconstructions
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(10, 10))

for i in range(5):
    for j in range(5):
        axes[i][j].imshow(reconstructions[i*5 + j].cpu().squeeze(0), cmap='gray')
        axes[i][j].axis('off')
plt.tight_layout(pad=0.)